# User Example: SAR Coherence

__Description & purpose__: This Notebook outlines a use case for generating Sentinel 1 SAR (Synthetic Aperture Radar) coherence imagery. It walks the user through the science and outlines how the eoap-gen tool has been used to create a workflow that is usable on the EODH. 

__Author(s)__: Alastair Graham, Dusan Figala

__Date created__: 2024-11-08

__Date last modified__: 2025-03-11

__Licence__: This file is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/).  Any included code is released using the [BSD-2-Clause](https://www.tldrlegal.com/license/bsd-2-clause-license-freebsd) license.


<span style="font-size:0.75em;">
Copyright (c) , All rights reserved.</span>

<span style="font-size:0.75em;">
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="font-size:0.75em;">
Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# Background

A requirement to process coherence from Sentinel 1 data was articulated.  

Coherence data are a measure of the similarity between two complex radar signals acquired over the same area. In this use case those radar signals are captured by the Sentinel-1 Synthetic Aperture Radar (SAR) sensor using its Single Look Complex (SLC) mode. Coherence is calculated by comparing the phase information of the radar signals across consecutive or closely spaced acquisitions. High coherence indicates strong similarity in phase, which typically occurs when the surface properties remain stable between acquisitions, such as in urban areas or bare soil. Conversely, low coherence suggests changes in the surface, such as vegetation growth, soil moisture variations, or physical displacement caused by events such as earthquakes or landslides. The coherence value, ranging between 0 and 1, provides valuable information about the temporal stability of the observed area.

Coherence data derived from Sentinel-1 SLC imagery are crucial in various remote sensing applications, including land-use classification, vegetation monitoring, and deformation analysis. In particular, coherence is a key component of interferometric SAR (InSAR) techniques, which rely on phase differences to measure ground displacement with millimeter precision. Coherence can be used to monitor dynamic changes in the environment, such as detecting deforestation, tracking agricultural practices, or studying snow cover variations. By leveraging Sentinel-1's high spatial resolution, regular revisit times, and all-weather imaging capabilities, coherence data provide a robust tool for understanding and managing natural and human-induced changes on the Earth's surface.

"The Sentinel 1 Level 1 SLC products are images in the slant range by azimuth imaging plane, in the image plane of satellite data acquisition. Each image pixel is represented by a complex (I and Q) magnitude value and therefore contains both amplitude and phase information. Each I and Q value is 16 bits per pixel. The processing for all SLC products results in a single look in each dimension using the full available signal bandwidth. The imagery is geo-referenced using orbit and attitude data from the satellite." [ref](https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel1.html)

Information on Sentinel 1 and the different products that are available at source can be found on the Copernicus Data Space Ecosystem (CDSE) [website](https://dataspace.copernicus.eu/explore-data/data-collections/sentinel-data/sentinel-1) and [here](https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel1.html)

# The Scientific Process


Before creating a workflow on the EODH platform it was important to step through the scientific process to make sure that the correct tools and data were being considered. To create the coherence outputs we use a relatively new Python package called [eo-tools](https://eo-tools.readthedocs.io/en/latest/) which calls another powerful packaged called [pygmtsar](https://github.com/AlexeyPechnikov/pygmtsar) (see also [https://insar.dev/](https://insar.dev/)).

As part of this discovery phase, and in line with the how-to tutorial on the eo-tools website, there is a need to access data on the CDSE. If you want to do this you will need a free account on that platform and will need to set up an `eodag.yaml` file as explained [here](https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html). 

The following tutorial will demonstrate how to process a single SLC pair.

First we need to import the required packages

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

from itertools import combinations

# general geospatial tools
import geopandas as gpd
import shapely
from shapely.geometry import shape
import folium

# eo-tools and related packages
import eo_tools as eo
from eo_tools.S1.process import process_insar
from eodag import EODataAccessGateway

import pyeodh

Make sure that a suitable directory is set as the working directory for this exploratory work. Set the following for your system.

In [4]:
# Set download dirs
data_dir = "/home/al/Downloads/eotools"

The first thing to do is set an Area of Interest (AOI). The simplest way to do this is to supply a geojson file and we have provided in the repository a small area near Thetford, UK.  

In [5]:
# You can supply an suitable geojson file

# AOI around Thetford
file_aoi = f"data/thetfordaoi.geojson"
shp = gpd.read_file(file_aoi).geometry[0]

# Alternatively you can set the AOI in code using the following format. The numbers are in decimal degrees and should be set for your AOI 
# bbox = {
#     "lonmin": 0.1,
#     "latmin": 52.1,
#     "lonmax": 0.9,
#     "latmax": 52.9,
# }
# shp = shapely.box(bbox["lonmin"], bbox["latmin"], bbox["lonmax"], bbox["latmax"])

## Scientific method

We start with searching the data on Copernicus Data Space Ecosystem.

Need to create an access point to do the search.

In [6]:
dag = EODataAccessGateway()
# make sure cop_dataspace will be used
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

INFO:eodag.config:Loading user configuration from: /home/al/.config/eodag/eodag.yml
INFO:eodag.core:usgs: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:aws_eos: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:meteoblue: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:hydroweb_next: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:wekeo: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:creodias_s3: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:Locations configuration loaded from /home/al/.config/eodag/locations.yml


In [7]:
# Run search using the bbox set earlier

search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-09-03",
    "end": "2023-09-17",
    "geom": shp,
}

results, _ = dag.search(**search_criteria)

INFO:eodag.core:Searching product type 'S1_SAR_SLC' on provider: cop_dataspace
INFO:eodag.search.qssearch:Sending search request: http://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel1/search.json?startDate=2023-09-03&completionDate=2023-09-17&geometry=POLYGON ((0.1085 52.5485, 0.6857 52.5580, 0.6778 52.4539, 0.1057 52.4464, 0.1085 52.5485))&productType=SLC&maxRecords=20&page=1&exactCount=1
INFO:eodag.core:Found 5 result(s) on provider 'cop_dataspace'


If we want to we can display all of the returned image file outlines on a map

In [8]:
eo.util.explore_products(results, shp)

We are only interested in image pairs. The S1 SLC file name doesn't make it clear which these are so the following code finds image footprints with 98% overlap

In [9]:
# Find overlaps
data = []
for item in results:
    id = item.properties["id"]
    geom = shape(item.geometry)
    data.append({"id": id, "geometry": geom})

gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")  # Assuming WGS84

# 98% overlap
threshold = 0.98

overlaps = []
for (idx1, row1), (idx2, row2) in combinations(gdf.iterrows(), 2):
    intersection = row1["geometry"].intersection(row2["geometry"])
    if not intersection.is_empty:
        # Calculate overlap ratio as the area of intersection divided by the area of the smaller polygon
        overlap_ratio = intersection.area / min(
            row1["geometry"].area, row2["geometry"].area
        )
        if overlap_ratio >= threshold:
            overlaps.append((row1["id"], row2["id"], overlap_ratio))

overlap_ids = [entry[:-1] for entry in overlaps]
overlap_ids

[('S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F',
  'S1A_IW_SLC__1SDV_20230916T174209_20230916T174236_050356_061016_8033')]

The data files are large (8GB). when running this locally use a tool such as `wget` to download the Sentinel 1 image pair that you want  

In [11]:
# Set download dirs
ids = [
    "S1A_IW_SLC__1SDV_20241016T174206_20241016T174233_056131_06DE72_1B0C",
    "S1A_IW_SLC__1SDV_20241028T174206_20241028T174233_056306_06E564_F046"
]
    #"S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F",
    #"S1A_IW_SLC__1SDV_20230916T174209_20230916T174236_050356_061016_8033",
#]

primary_dir = f"{data_dir}/{ids[0]}.zip"
secondary_dir = f"{data_dir}/{ids[1]}.zip"
outputs_prefix = f"{data_dir}/res/test-full-processor"

The next cell runs `eo-tools` on the SAR SLC data pair. There are a multitude of configuration parameters but the most important one for this application is to ensure that `write_coherence` is set to `True`. This will generate an image of coherence in the project path set earlier in the Notebook. To effectively set the remainder of the parameters you will need some level of competence in SAR interferometry. For the purposes of this demonstration we leave them on the default values.

In [12]:
out_dir = process_insar(
    dir_prm=primary_dir,
    dir_sec=secondary_dir,
    outputs_prefix=outputs_prefix,
    aoi_name=None,
    shp=shp,
    pol="vv",
    subswaths=["IW1", "IW2", "IW3"],
    write_coherence=True,
    write_interferogram=True,
    write_primary_amplitude=False,
    write_secondary_amplitude=False,
    apply_fast_esd=True,
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    boxcar_coherence=[3, 3],
    filter_ifg=True,
    multilook=[1, 4],
    warp_kernel="bicubic",
    clip_to_shape=True,
)

INFO:eo_tools.S1.process:---- Processing subswath IW1 in VV polarization
INFO:eo_tools.S1.core:S1IWSwath Initialization:
INFO:eo_tools.S1.core:- Read metadata file /home/al/Downloads/eotools/S1A_IW_SLC__1SDV_20241016T174206_20241016T174233_056131_06DE72_1B0C.zip/S1A_IW_SLC__1SDV_20241016T174206_20241016T174233_056131_06DE72_1B0C.SAFE/annotation/s1a-iw1-slc-vv-20241016t174207-20241016t174232-056131-06de72-004.xml
INFO:eo_tools.S1.core:- Read calibration file /home/al/Downloads/eotools/S1A_IW_SLC__1SDV_20241016T174206_20241016T174233_056131_06DE72_1B0C.zip/S1A_IW_SLC__1SDV_20241016T174206_20241016T174233_056131_06DE72_1B0C.SAFE/annotation/calibration/calibration-s1a-iw1-slc-vv-20241016t174207-20241016t174232-056131-06de72-004.xml
INFO:eo_tools.S1.core:- Set up raster path zip:///home/al/Downloads/eotools/S1A_IW_SLC__1SDV_20241016T174206_20241016T174233_056131_06DE72_1B0C.zip/S1A_IW_SLC__1SDV_20241016T174206_20241016T174233_056131_06DE72_1B0C.SAFE/measurement/s1a-iw1-slc-vv-20241016t17420

## Outputs

The following figures show, in order:
* The AOI
  * This covers an area between the Fenland town of March to the west and Thetford Forest to the east. The 100 Foot Drain complex is covered in the western portion of the AOI
* The full coherence image
  * Low coherence values are represented in dark blue and high values in yellow, with a gradient between them.
* A portion of the coherence image near the town of March 

![AOI](../img/co/co1.png){width=1000px}

![Coherence](../img/co/co2.png){width=1000px}

![Close up of coherence](../img/co/co3.png){width=900px}


## Using pyeodh

We can use `pyeodh` to find data on the Hub. The following cells demonstrate how this might be done.

In [2]:
# Connect to the Hub
# base_url can be changed to optionally specify a different server, such as test.eodatahub

client = pyeodh.Client(
    base_url="https://staging.eodatahub.org.uk"
).get_catalog_service()

The following cell prints out the available properties for each item. It is possible to ammend this to print specific properties for each item (up to the specified limit).

In [ ]:
# Now we want to access the first few items  
sentinel = client.get_catalog("supported-datasets/catalogs/ceda-stac-catalogue").get_collection('sentinel1')
sentinel.get_items()

lim = 1 # increase this number to access more items

for i, item in enumerate(sentinel.get_items()):
    if i >= lim:
        break
    print(item.id, item.properties) # comment this line and uncomment the next to just print id and datetime for each item
#    print(item.id, item.properties['datetime'])  

neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.31.S1A_IW_SLC__1SDV_20241031T233334_20241031T233407_056353_06E745_B4ED {'datetime': '2024-10-31T23:33:50Z', 'created': '2024-11-19T13:58:04.869039Z', 'updated': '2024-11-19T13:58:04.869039Z', 'start_datetime': '2024-10-31T23:33:34Z', 'end_datetime': '2024-10-31T23:34:07Z', 'platform': 'sentinel1a', 'instrument_mode': 'IW', 'processing_level': 'L1', 'product_type': 'SLC', 'processing_software': 'IPF', 'product_version': 'v3', 'orbit': '056353', 'Instrument Family Name': 'Synthetic Aperture Radar', 'Instrument Family Name Abbreviation': 'SAR', 'Platform Number': 'A', 'NSSDC Identifier': '2014-016A', 'Start Relative Orbit Number': '106', 'Start Orbit Number': '56353', 'Instrument Mode': 'IW', 'Slice Number': '18', 'Total Slices': '18', 'Polarisation': ['VV', 'VH'], 'Processing Version': '003.80', 'Orbit Direction': 'DESCENDING', 'EPSG': '4326', 'instance_id': 'neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.31.S1A_IW_SLC__1SDV_20241031T233334_

Now we want to search for SAR data that fall across the Thetford site for a given timeframe.

In [5]:
# To find specific imagery for the Thetford site we need to add the intersects parameter. We set this to be our AOI point.
thet_pnt = shapely.Point(0.6715892933273722, 52.414471075812315) # a site near Thetford

# Set the date range as you need
items = client.search(
    collections=['sentinel1'],
    catalog_paths=["supported-datasets/catalogs/ceda-stac-catalogue"],
    intersects=thet_pnt,
    query=[
        'start_datetime>=2024-10-16',
        'end_datetime<=2024-11-30', 
    ],
)

# We can then count the number of items returned by the search 
#print('Number of items found: ', items.total_count)

total_items = sum(1 for _ in items)
print(f"Total items: {total_items}")

Total items: 9


The next cell creates a formatted list of id, bounding box and a data URL for each of the items.

It then creates a Shapely polygon and uses this to look for other images that have 98% overlap 

In [ ]:
# Create the formatted list
data = [
    (item.id, 
     item.bbox,
     item.assets.get("data_zip")
) for item in items
]

# Convert to dictionary with shapely boxes
bboxes = {rec[0]: shapely.geometry.box(*rec[1]) for rec in data}

# Find IDs where overlap is at least 90%
overlapping_ids = set()

for id1, bbox1 in bboxes.items():
    for id2, bbox2 in bboxes.items():
        if id1 != id2:  # Avoid self-comparison
            intersection_area = bbox1.intersection(bbox2).area
            min_area = min(bbox1.area, bbox2.area)
            if min_area > 0 and (intersection_area / min_area) >= 0.98:
                overlapping_ids.add(id1)

print("IDs with at least 98% overlap:", overlapping_ids)


IDs with at least 98% overlap: {'neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.18.S1A_IW_SLC__1SDV_20241018T055407_20241018T055442_056153_06DF4E_19D4', 'neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.18.S1A_IW_SLC__1SDV_20241018T061507_20241018T061535_056153_06DF50_619B', 'neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.30.S1A_IW_SLC__1SDV_20241030T055407_20241030T055442_056328_06E63C_37A1', 'neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.26.S1A_IW_SLC__1SDV_20241026T181923_20241026T181948_056277_06E441_9ADB', 'neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.25.S1A_IW_SLC__1SDV_20241025T060655_20241025T060722_056255_06E360_E4C0', 'neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.30.S1A_IW_SLC__1SDV_20241030T061507_20241030T061534_056328_06E63E_21E9', 'neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.28.S1A_IW_SLC__1SDV_20241028T174206_20241028T174233_056306_06E564_F046', 'neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.16.S1A_IW_SLC__1SDV_20241016T174206_20241016T174233_056131_06DE72_1B0C'}


Finally, we want to plot the overlapping items on a map. We also create a FeatureGroup to allow the bounding boxes to be swithched on and off: this helps us understand which images are direct image pairs. as before, we need to make a note of those image pairs and then run the `print(data)` cell to find the corresponding URL to the data.  

In [ ]:
# Compute the center of the first bounding box for map initialization
first_bbox = list(bboxes.values())[0]
center = first_bbox.centroid.y, first_bbox.centroid.x  # (lat, lon)

# Create a folium map centered on the first bbox
m = folium.Map(location=center, zoom_start=6)

# Add each bbox as a separate FeatureGroup (allows toggling)
for item_id, bbox in bboxes.items():
    feature_group = folium.FeatureGroup(name=f"Polygon {item_id}")  # Group for toggling
    coords = [(y, x) for x, y in bbox.exterior.coords]  # Convert (x, y) → (lat, lon)
    
    folium.Polygon(
        locations=coords,
        color="blue",
        fill=True,
        fill_opacity=0.4,
        popup=item_id
    ).add_to(feature_group)
    
    feature_group.add_to(m)  # Add the group to the map

# Add LayerControl to allow toggling
folium.LayerControl().add_to(m)

# Display the map
m

In [9]:
print(data)

[('neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.30.S1A_IW_SLC__1SDV_20241030T061507_20241030T061534_056328_06E63E_21E9', [-2.245931, 51.279499, 2.000001, 53.306957], <Asset href=https://dap.ceda.ac.uk/neodc/sentinel1a/data/IW/L1_SLC/IPF_v3/2024/10/30/S1A_IW_SLC__1SDV_20241030T061507_20241030T061534_056328_06E63E_21E9.zip>), ('neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.30.S1A_IW_SLC__1SDV_20241030T055407_20241030T055442_056328_06E63C_37A1', [-179.947861, 50.773228, 177.047455, 53.279991], <Asset href=https://dap.ceda.ac.uk/neodc/sentinel1a/data/IW/L1_SLC/IPF_v3/2024/10/30/S1A_IW_SLC__1SDV_20241030T055407_20241030T055442_056328_06E63C_37A1.zip>), ('neodc.sentinel1a.data.IW.L1_SLC.IPF_v3.2024.10.28.S1A_IW_SLC__1SDV_20241028T174206_20241028T174233_056306_06E564_F046', [-0.871057, 51.935623, 3.405478, 53.953266], <Asset href=https://dap.ceda.ac.uk/neodc/sentinel1a/data/IW/L1_SLC/IPF_v3/2024/10/28/S1A_IW_SLC__1SDV_20241028T174206_20241028T174233_056306_06E564_F046.zip>), ('neodc.sentin

Once you have the data URLs for the image pairs you are interested in, scroll up to follow the cells that use eo-tools to download and process the data.

# Generating the workflow

When moving this workflow into the EODH platform it is best practice to use `eoap-gen`. The tool requires a configuration file which is provided [here](https://github.com/EO-DataHub/user-workflows/blob/main/S1-coherence/eoap-gen-config.yml) and provided below. 

Running this configuration file generates the folder structure available [here](https://github.com/EO-DataHub/user-workflows/tree/main/S1-coherence/eoap-gen-out/cli) which can be executed using the EODH workflow runner. 

# Commercial data and processing services

Commercial SLC data can also be used to generate coherence datasets. Commercial Airbus Synthetic Aperture Radar (SAR) data, such as those from the TerraSAR-X or TanDEM-X missions, can be used to generate coherence products by leveraging their high-resolution and high-quality Single Look Complex (SLC) imagery. These Airbus supplied SAR data provide precise orbital control and excellent temporal and spatial resolution, which are critical for ensuring accurate phase comparisons. By processing pairs of TerraSAR-X or TanDEM-X images, coherence maps can be derived to assess temporal stability and surface dynamics. Additionally, the fine spatial resolution of Airbus SAR data enhances the detail and accuracy of coherence products, making them particularly valuable for localized studies and detailed environmental or industrial assessments.

The [Airbus One-ATLAS Developer portal](https://www.geoapi-airbusds.com/index.html) provides access to such data and case-studies related to interferrometry and coherence are linked to in the following bullet points:
* High Resolution SAR data: https://intelligence.airbus.com/imagery/our-optical-and-radar-satellite-imagery/radar-constellation/
* Interferometry case studies: https://intelligence.airbus.com/search/?q=interferometry